# Policy Gradient 方法

## 什么是 Policy Gradient 方法

Policy Gradient 方法是通过直接 **建模和优化 policy** 来完成强化学习任务的方法。而优化过程中, 简单来说, 我们最大化对 return 的估计, 优化的方法是 **梯度上升**。

例如, 一个基本的优化目标函数是:

$$ J(\theta) = \sum_s{d^\pi(s) V^\pi(s)} = \sum_s{d^\pi(s) \sum_a{\pi(a|s) Q^\pi(s,a)}} $$

其中, $d^\pi(s)$ 是状态 $s$ 在 policy $\pi$ 下的稳态分布, 即:

$$ d^\pi(s) = \lim_{t \to \infty} P(s_t = s|s_0, \pi_{\theta}) $$

不同的 Policy Gradient 方法会定义不同的 **优化目标函数** 以达到正则化的目的, 或者使用不同的方法来估计目标函数。

## 相比于 Value-based 方法, Policy-based 方法有什么优势？

1. **连续问题**: 更适合连续动作空间或连续状态空间的问题
2. **收敛速度往往更快**: 直接优化 policy, 而不是通过优化 value function 来间接更新 policy, 通常收敛速度更快
3. **stochasticity, 更好的 exploration**: value-based 方法学习到一个 value function 后, 最优策略是一个 deterministic policy, 需要人为引入例如 $\epsilon$-greedy 策略来(低效地)增加 exploration 。而 policy-based 方法天然就是一个 stochastic policy, 这样可以更好地增加 exploration。

## Policy Gradient 方法的关键

根据上面给出的优化目标函数, 我们可以看到: Policy gradient $\nabla J(\theta)$ 不仅依赖于动作 a 的选择m 还依赖于稳态分布 $d^{\pi}(s)$(难以估计)。

不过, 好在我们可以通过 **策略梯度定理** 来估计这个目标函数:

$$ \nabla_{\theta} J(\theta) = \nabla_{\theta} \sum_s d^{\pi}(s) \sum_a \pi(a|s) Q^{\pi}(s,a) \propto \sum_s d^{\pi}(s) \sum_a Q^{\pi}(s, a) \nabla_{\theta} \pi_{\theta}(a|s) $$

基于策略梯度定理, 无数的具体的 Policy Gradient 方法都可以从中派生出来。

## Policy Gradient 方法枚举

### REINFORCE

REINFORCE 是最基本的 Policy Gradient 方法, 它通过 **Monte-Carlo** 方法来估计 return, 从而估计目标函数。其梯度公式如下:

$$ \nabla_{\theta} J(\theta) = \mathbb{E}_{\pi} [Q^{\pi}(s,a) \nabla_{\theta} \log \pi_{\theta}(a|s)] $$

由于 $Q^{\pi}(S_t,A_t) = \mathbb{E}_{\pi}[G_t|S_t = s, A_t = a]$, 因此:

$$ \nabla_{\theta} J(\theta) = \mathbb{E}_{\pi}[G_t \nabla_{\theta} \log \pi_{\theta}(A_t|S_t)] $$

这样, 我们就可以通过采样得到的 trajectory 来获得 $G_t$ 的估计, 从而估计目标函数。具体算法流程如下:

1. 随机初始化 policy $\pi_{\theta}$
2. 根据 $\pi_{\theta}$ 采样 trajectory $S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, \cdots$
3. 对于 $t=0,1,2,\cdots, T$:
    i. 计算 $G_t = \sum_{k=t}^{\infty} \gamma^{k-t} R_k$
   ii. 更新参数 $\theta$: $\theta \leftarrow \theta + \alpha \gamma^{t} G_t \nabla_{\theta} \log \pi_{\theta}(A_t|S_t)$

p.s. 公式当然可以进一步化简为:
$$\theta \leftarrow \theta + \alpha \gamma^{t} G_t \frac{\nabla_{\theta} \pi_{\theta}(A_t|S_t)}{\pi_{\theta}(A_t|S_t)}$$

不过更为广泛的 REINFORCE 算法会使用 **baseline** 来减少方差, 也就是说: 参数更新式中的 $G_t$ 会减去一个 baseline value 来减少梯度估计的方差, 例如: 使用 (state, action) value 减去 state value。这样, 我们就可以得到:
$$\theta \leftarrow \theta + \alpha \gamma^{t} \hat{B_t} \frac{\nabla_{\theta} \pi_{\theta}(A_t|S_t)}{\pi_{\theta}(A_t|S_t)}$$
其中, $\hat{B_t} = G_t - V(S_t)$

### Vanilla Actor-Critic 方法

我们看到在 Policy Gradient 中, 既有对策略的估计: $\pi_{\theta}(a|s)$, 也有对 value function 的估计: $Q^{\pi}(s,a)$。我们既然使用了神经网络来估计了 policy, 为什么不增加一个网络来估计 value 呢？这样，我们就引出了 **Actor-Critic** 方法。

一个简单的 Actor-Critic 方法的训练算法如下:

1. 人为选定超参数: $\alpha_{\theta}$, $\alpha_w$, 随机初始化 $s$, $\theta$, $w$, 采样 $a \sim \pi_{\theta}(a|s)$
2. 对于 $t = 1 \cdots T$:
    i. 采样 $s^{'}, r_{t+1} \sim p(s^{'}, r_{t+1}|s, a)$
   ii. 采样 $a^{'} \sim \pi_{\theta}(a^{'}|s^{'})$
   iii. 更新策略网络 $\pi_{\theta}$: $\theta \leftarrow \theta + \alpha_{\theta} Q_w(s,a) \nabla_{\theta} \log \pi_{\theta}(a|s)$
    v. 计算 action-value 在 $t$ 时刻的 TD error: $\delta_t = r_t + \gamma Q_w(s^{'}, a^{'}) - Q_w(s, a)$, 更新价值网络 $Q_w$: $w$: $w \leftarrow w + \alpha_w \delta_t \nabla_{w} Q_w(s, a)$
   vi. $s \leftarrow s^{'}, a \leftarrow a^{'}$

**reference**

1. [https://lilianweng.github.io/lil-log/2018/04/08/policy-gradient-algorithms.html#actor-critic](https://lilianweng.github.io/lil-log/2018/04/08/policy-gradient-algorithms.html#actor-critic)
2. [https://lilianweng.github.io/posts/2018-02-19-rl-overview/](https://lilianweng.github.io/posts/2018-02-19-rl-overview/)